In [1]:
import warnings

warnings.simplefilter("ignore")

In [2]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

Diversos algoritmos son sensibles a la escala en la que viene cada feature. **Re-escalarlos** puede traer significativas mejoras de rendimiento.
Existen distintas estrategias de escalamiento de tus features, pero la más común es la estandarización donde convertimos la variable para que la distribución de esta siga una distribución que es Gaussiana de media 0 y de desviación estandar 1.

In [3]:
%%time

from sklearn.model_selection import train_test_split

X = pd.read_csv('X_openingg.csv')
y = X['worldwide_gross']
X = X.drop(['worldwide_gross','opening_gross','screens'], axis=1)           
           
X_train, X_test, y_train, y_test = train_test_split(X,y)  

Wall time: 1.65 s


In [4]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

scaler.fit(X_train)

StandardScaler(copy=True, with_mean=True, with_std=True)

In [5]:
#7 promedios que corresponden a cada una de las features

scaler.mean_

array([5.06886586e+07, 2.00617057e+03, 2.15537241e+00, 1.10625225e+02,
       1.40008330e+04, 5.30587045e+07, 6.33525526e+00])

In [6]:
#corresponde a la desviación estándar de cada feature

scaler.scale_

array([4.40386303e+07, 5.50967252e+00, 2.52518375e-01, 1.97067715e+01,
       2.37924215e+04, 6.83627419e+07, 1.01377317e+00])

In [7]:
#X antes de fit_transform (con valores grandes)

X

,production_budget,title_year,aspect_ratio,duration.1,cast_total_facebook_likes,budget,imdb_score
0,425000000.0,2009.0,1.780000,178.0,4834.0,2.370000e+08,7.9
1,300000000.0,2007.0,2.350000,169.0,48350.0,3.000000e+08,7.1
2,300000000.0,2015.0,2.350000,148.0,11700.0,2.450000e+08,6.8
3,275000000.0,2012.0,2.350000,164.0,106759.0,2.500000e+08,8.5
4,275000000.0,2013.0,2.350000,150.0,45757.0,2.150000e+08,6.5
...,...,...,...,...,...,...,...
2216,10800000.0,2006.0,2.350000,118.0,378.0,1.500000e+07,6.4
2217,10600000.0,2013.0,2.350000,98.0,2271.0,7.000000e+06,5.5
2218,10500000.0,2007.0,2.350000,97.0,20312.0,1.000000e+07,6.5
2219,10100000.0,2010.0,2.157506,144.0,1121.0,5.329615e+07,5.3


In [8]:
#X luego de escalar los datos, estamos mas centrados en 0 y en escalas más parecidas (no tan grandes ni tan chicos)

scaler.transform(X_train)

array([[-0.24271097,  0.69503758,  0.77074626, ..., -0.41012358,
        -0.19102078,  0.06386512],
       [-0.6968577 ,  0.69503758,  0.77074626, ..., -0.4293734 ,
        -0.4835778 , -0.92254883],
       [ 1.11972923,  0.69503758,  0.77074626, ..., -0.01008023,
         0.68665027,  0.55707209],
       ...,
       [ 0.66558249, -0.21245738, -1.20930767, ..., -0.51263521,
         0.320954  , -0.03477628],
       [-0.07240594,  1.78403152,  0.77074626, ..., -0.50002615,
        -0.41043855,  0.95163767],
       [-0.24271097, -0.93845334, -1.20930767, ...,  0.65076045,
        -0.19102078,  0.3597893 ]])

In [9]:
#vamos a crear un x_train escalado y un x_test escalado. vamos a utilizar el mismo scaler para transformar los datos de test.
#tanto los datos de entrenamiento como los de test deben ser escalados con el mismo scaler.

X_train_scaled, X_test_scaled = (scaler.transform(X_train), scaler.transform(X_test))



In [10]:
#vamos a entrenar el modelo sin escalar y el modelo escalado

from sklearn.linear_model import Lasso

model = Lasso()
model_scaled = Lasso()


model.fit(X_train, y_train)

model_scaled.fit(X_train_scaled, y_train)

Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [11]:
#vamos a ver los scores

print(model.score(X_test, y_test))
print(model_scaled.score(X_test_scaled, y_test))

0.5675751302293216
0.5675751318046671


Los modelos de regresión no se ven afectados por el escalamiento de las features. Los de clasificación sí. 
Las regresiones no necesitan reescalamiento, pero si las clasificaciones.

Simplificar las transformaciones con pipelines.
Para hacer tu código más reproducible, y para evitar tener que aplicar multiples veces una misma transformación. utilizar  sklearn.pipeline.make_pipeline  que permite encadenar transformaciones a tus modelos.

In [12]:
from sklearn.pipeline import make_pipeline

model_scaled = make_pipeline(StandardScaler(),
                            Lasso())

model_scaled.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('lasso',
                 Lasso(alpha=1.0, copy_X=True, fit_intercept=True,
                       max_iter=1000, normalize=False, positive=False,
                       precompute=False, random_state=None, selection='cyclic',
                       tol=0.0001, warm_start=False))],
         verbose=False)

In [13]:
print(model_scaled.score(X_test, y_test))

0.5675751318046671


Crear nuevas features de forma automática

In [14]:
A = np.arange(6).reshape(3,2)
A

array([[0, 1],
       [2, 3],
       [4, 5]])

In [15]:
from sklearn.preprocessing import PolynomialFeatures

transformer = PolynomialFeatures(2)
transformer.fit_transform(A)

array([[ 1.,  0.,  1.,  0.,  0.,  1.],
       [ 1.,  2.,  3.,  4.,  6.,  9.],
       [ 1.,  4.,  5., 16., 20., 25.]])

PolynomialFeatures transforma una matriz $(A1,A2)$ a $(1,A1,A2,A1^2,A1\cdot A2,A2^2)$

In [16]:
X.shape

(2221, 7)

In [17]:
transformer = PolynomialFeatures(2)
transformer.fit_transform(X).shape

(2221, 36)

In [20]:
#score bajo porque aumentamos demasiado las dimensiones y no habia correlacion con las features cuadráticas de grado 2 (las que creamos)

model_poly = make_pipeline(PolynomialFeatures(2),
                           Lasso())

model_poly.fit(X_train, y_train)
model_poly.score(X_test,y_test)

0.633121549596567

In [21]:
model = Lasso()
model.fit(X_train,y_train)
model.score(X_test,y_test)

0.5675751302293216

Crear features categóricas
En terminos de Machine Learning a las features que pueden tomar un número finito de valores se les llama categóricas. Ejemplos para esto són: género, páis, grado académico, etc.

Para no introducir información falsa o erronéa en nuestro modelos existen formas más inteligentes de encodear nuestros datos.

Encoding one-hot

Este encoding consiste en asignarle una columna a cada categoría y rellenarla con 0 y 1 de la forma siguiente:

In [22]:
d = pd.DataFrame([['Chile','Colombia','Colombia','Venezuela'],['hombre','mujer','hombre','mujer']])
d = d.T
d.columns = pd.Index(['pais','genero'])
d

,pais,genero
0,Chile,hombre
1,Colombia,mujer
2,Colombia,hombre
3,Venezuela,mujer


In [23]:
#para una solucuón rapida y que se puede ocupar en todos los casos se usa el metodo de pandas get_dumies
pd.get_dummies(d)

,pais_Chile,pais_Colombia,pais_Venezuela,genero_hombre,genero_mujer
0,1,0,0,1,0
1,0,1,0,0,1
2,0,1,0,1,0
3,0,0,1,0,1


Sklearn también ofrece un objeto OneHotEncoder pero es un poco más díficil de utilizar, así que por criterios pedagogicos hemos elegido pd.get_dummies. Sin embargo el objeto de sklearn tiene la ventaja de ser pipeable, por lo que es bueno considerarlo para ciertos casos de uso.

Cuantas columnas generaríamos con un one-hot encoding de nuestras features categóricas?

In [24]:
movies_obj = pd.read_csv('movies_obj.csv')

In [25]:
movies_obj.apply(pd.Series.nunique).sort_values()

color                2
content_rating      18
language            47
country             65
genres             914
actor_1_name      2097
director_name     2398
actor_2_name      3032
actor_3_name      3521
plot_keywords     4760
movie_title       4917
dtype: int64

Las features más informativas son las del casting. Si embargo haciendo un one-hot encoding de estas estaríamos aumentando la dimensión por 2000 y algo!!

Encoding Binario

Esta técnica no es canónica por lo que tendremos que buscarla en otra librería. Sin embargo el autor tuvo la buena idea de hacer su API compatible con la de sklearn, así que no tendremos ninguna dificultad en usarla.

$$ Categoria \rightarrow Numero \rightarrow Binario \rightarrow Columnas $$

In [ ]:
!pip install category_encoders

In [26]:
categoricals = pd.read_csv('categoricals.csv').set_index('Unnamed: 0')

In [27]:
categoricals.head(2)

,actor_1_name,director_name
Unnamed: 0,,
0,CCH Pounder,James Cameron
1,Doug Walker,Doug Walker


In [28]:
#indexo bien los datos, reseteo el index y lo borro. los valores faltantes los reemplazo con 0. fillna metodo pandas

categoricals = categoricals.reset_index(drop=True).fillna(0)

In [30]:
categoricals.head(2)

,actor_1_name,director_name
0,CCH Pounder,James Cameron
1,Doug Walker,Doug Walker


In [31]:
X_binec = pd.concat([X, categoricals], axis=1)

In [32]:
X_binec.head()

,production_budget,title_year,aspect_ratio,duration.1,cast_total_facebook_likes,budget,imdb_score,actor_1_name,director_name
0,425000000.0,2009.0,1.78,178.0,4834.0,237000000.0,7.9,CCH Pounder,James Cameron
1,300000000.0,2007.0,2.35,169.0,48350.0,300000000.0,7.1,Doug Walker,Doug Walker
2,300000000.0,2015.0,2.35,148.0,11700.0,245000000.0,6.8,Johnny Depp,Gore Verbinski
3,275000000.0,2012.0,2.35,164.0,106759.0,250000000.0,8.5,Christoph Waltz,Sam Mendes
4,275000000.0,2013.0,2.35,150.0,45757.0,215000000.0,6.5,Tom Hardy,Christopher Nolan


In [33]:
import category_encoders as ce
encoder = ce.BinaryEncoder(cols=['actor_1_name', 'director_name'])

In [34]:
encoder.fit_transform(X_binec).shape

(4104, 31)

In [35]:
X_binec = encoder.fit_transform(X_binec)

In [36]:
y = pd.read_csv('y.csv')

In [37]:
y.shape

(4103, 2)

In [38]:
XX = pd.concat([X_binec,y], axis=1)

In [39]:
XX.head()

,production_budget,title_year,aspect_ratio,duration.1,cast_total_facebook_likes,budget,imdb_score,actor_1_name_0,actor_1_name_1,actor_1_name_2,...,director_name_4,director_name_5,director_name_6,director_name_7,director_name_8,director_name_9,director_name_10,director_name_11,0,2783918982.0
0,425000000.0,2009.0,1.78,178.0,4834.0,237000000.0,7.9,0,0,0,...,0,0,0,0,0,0,0,1,1.0,2.058662e+09
1,300000000.0,2007.0,2.35,169.0,48350.0,300000000.0,7.1,0,0,0,...,0,0,0,0,0,0,1,0,2.0,9.634204e+08
2,300000000.0,2015.0,2.35,148.0,11700.0,245000000.0,6.8,0,0,0,...,0,0,0,0,0,0,1,1,3.0,8.796209e+08
3,275000000.0,2012.0,2.35,164.0,106759.0,250000000.0,8.5,0,0,0,...,0,0,0,0,0,1,0,0,4.0,1.084439e+09
4,275000000.0,2013.0,2.35,150.0,45757.0,215000000.0,6.5,0,0,0,...,0,0,0,0,0,1,0,1,5.0,2.600021e+08


In [40]:
XX= XX.drop('0', axis=1)


In [41]:
#borro valores nulos

XX.dropna(inplace=True)

In [42]:
y = XX['2783918982.0']
X_binec = XX.drop('2783918982.0', axis=1)           
           

In [43]:
Xb_train, Xb_test, y_train, y_test = train_test_split(X_binec,y)


In [44]:
X_train, X_test = (Xb_train[X.columns],Xb_test[X.columns])


In [45]:
model_binenc = Lasso()
model = Lasso()

In [46]:
model_binenc.fit(Xb_train,y_train)
model.fit(X_train,y_train)

Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False)

In [47]:
print(model_binenc.score(Xb_test,y_test))
print(model.score(X_test,y_test))


0.49647990455896823
0.5023115603874397



Aumentamos el rendimiento de nuestro algoritmo pero no de forma significativa. Mantengamos entonces la dimensionalidad de nuestro espacio de features baja, y vamos a buscar modelos más complejos.

Conocimiento experto:

Una grán parte del diseño de las features pasa por un **conocimiento espécifico del dominio en el que se esta trabajando**.
Por ejemplo para analizar una imagen nuestro cerebro no se concentra en los millones de pixeles de una imagen, pero sólo en algunos relevantes como los de los contornos. Durante un buen tiempo **los sistemas de visión de computadores encodeaban features que traducían este conocimiento experto (contornos).**
Una de las únicas formas de obtener este conocimiento de forma sistemática es ir a bucear en repositorios de papers de Machine Learning como Arxiv, y estudiar la investigación que se ha hecho sobre el dominio específico.

Contamos con la base de datos de ganancias de las péliculas el primer fin de semana de exhibición, así como la cantidad de cines en la que fue estrenada.



In [48]:
pd.read_csv('opening_df.csv').head()

,Unnamed: 0,movie_title,opening_gross,screens
0,0,10 Days in a Madhouse,2451.0,10.0
1,1,10 Things I Hate About You,8330681.0,2271.0
2,2,102 Dalmatians,19883351.0,2704.0
3,3,12 Rounds,5329240.0,2331.0
4,4,12 Years a Slave,923715.0,19.0
